In [339]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'merge_wiki_32'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/merge_wiki_32/outputs/results/run']


In [340]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_index_build_duration_ns',
       'result.inner_index_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [341]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name     BTree2048     BTree256      PGM1024       PGM256  \
spec.common_key                                                       
10               105582608.0  104155317.0  107238272.5  104753828.0   
20               109439588.5  108761898.0  100939565.0  107595273.0   
30               108360566.5  101238816.5  107570410.0  130819022.5   
40               116812327.0  100969683.0   98783582.5  122706460.5   
50               112701997.5  129588661.0   95854628.0  106234143.5   
60               108329181.5  100011916.0  107451212.5   98020604.5   
70               109174197.5  109197467.5  115708724.5   95586875.0   
80               119287339.0  100980165.0  106142527.5   98492262.0   
90               102496418.5  114288066.0  102685153.0   95906397.5   
100              111134083.0  105019907.0  101758950.5  109602370.5   

spec.algo_name   standard_merge  
spec.common_key                  
10                  103919547.5  
20                  103721807.5  
30         

In [342]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

          indexes  memory_bytes
0       BTree2048       20592.0
1        BTree256      131832.0
2         PGM1024         512.0
3          PGM256        2240.0
4  standard_merge       68952.0


In [343]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))
print(inner_index_build_duration)

          indexes  build_duration
0       BTree2048    3.538446e+07
1        BTree256    3.596422e+07
2         PGM1024    2.559569e+08
3          PGM256    2.672323e+08
4  standard_merge    3.156426e+08


In [344]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   BTree2048  BTree256  PGM1024  PGM256  standard_merge
spec.common_key                                                      
10                    7788      7788     7788    7788            7787
20                    7789      7789     7789    7789            7789
30                    7789      7789     7789    7789            7789
40                    7789      7789     7789    7789            7789
50                    7788      7788     7788    7788            7788
60                    7789      7789     7789    7789            7789
70                    7790      7790     7790    7790            7790
80                    7790      7790     7790    7790            7790
90                    7790      7790     7790    7790            7790
100                   7791      7791     7791    7791            7791
spec.algo_name   BTree2048  BTree256   PGM1024    PGM256  standard_merge
spec.common_key                                                         
10            

In [345]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 50])
print(result_checksum.loc[result_checksum['spec.common_key'] == 50]['result.checksum'].unique())

common_key: 10 checksum: ['612555EAE128E3B6AFB7803C14FA793B'] OK
common_key: 20 checksum: ['46DCE793FB9F10A874BBDBD5A4752235'] OK
common_key: 30 checksum: ['3200D47CF71EF720B63BAE20229CA63D'] OK
common_key: 40 checksum: ['E92AD445454F818B82A05F2CBB0E526A'] OK
common_key: 50 checksum: ['F1A096C0C6175DB0A3BFA2037E8CE09C'] OK
common_key: 60 checksum: ['E0C4A8C986B3201B5849ABF8B63CA3D2'] OK
common_key: 70 checksum: ['7E9F194924B2E4A1F02576CC6786931D'] OK
common_key: 80 checksum: ['5440BF8D56583135129138B441B0E558'] OK
common_key: 90 checksum: ['DE1E999793B23FFB34F73156CE1CDBB5'] OK
common_key: 100 checksum: ['AF44563DC06CCCAA3339F95E3EFBADE2'] OK
     spec.common_key  spec.algo_name                   result.checksum
8                 50       BTree2048  F1A096C0C6175DB0A3BFA2037E8CE09C
61                50       BTree2048  F1A096C0C6175DB0A3BFA2037E8CE09C
101               50       BTree2048  F1A096C0C6175DB0A3BFA2037E8CE09C
195               50       BTree2048  F1A096C0C6175DB0A3BFA2037E8